In [1]:
import numpy as np
from scipy import signal
import soundfile as sf
import matplotlib.pyplot as plt
from IPython.display import Audio

from dichotic_pitch import dichotic_pitch, lowpass, cos_window

# Set up plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = [12, 8]

# Common constants
samp_sec = 48000    # samples per second
f_nyquist = samp_sec/2
f_max = 1200    # lowpass filter cutoff

In [ ]:
# Simple Dichotic Pitch
sbr = 1
peak = 500     # Hertz: center frequency of DP signal
width = peak/20    # Hertz: width of rectangular DP signal filter
ts_sig = -0.6     # milliseconds: time-shift of the signal
ts_back = 0.0     # milliseconds: time-shift of the background
note_duration = 0.5    # seconds

dp = dichotic_pitch(sbr, peak, width, ts_sig, ts_back, samp_sec, note_duration)

# lowpass filter 
#dp = lowpass(dp, f_max/f_nyquist)

# cosine window (10 ms)
dp = cos_window(dp, round(samp_sec/100))

# normalize 
dp = dp / np.max(np.abs(dp))

# Play and save audio
sf.write('dp.wav', dp, samp_sec)
Audio(dp.T, rate=samp_sec)

In [ ]:
# Plot power spectrum
plt.figure(figsize=(10, 6))
f, pxx = signal.welch(dp[:, 0], samp_sec, nperseg=256)
plt.semilogy(f, pxx)
f, pxx = signal.welch(dp[:, 1], samp_sec, nperseg=256)
plt.semilogy(f, pxx, color='green')
plt.xlim(0, f_max+100)
plt.grid(True)
plt.show()

In [ ]:
# The effect of signal-to-background ratio
sbr = [5, 4, 3, 2, 1, 0.8, 0.5, 0.25, 0.1]
print(f"The SBR levels: {sbr}")
peak = np.array([300, 600, 900])  # play a harmonic complex
width = peak/20
num_notes = len(sbr)
ts_sig = 0.6
ts_back = -0.6
note_duration = 0.5

parts = []
for i in range(num_notes):
    parts.append(np.concatenate([
        dichotic_pitch(sbr[i], peak, width, ts_sig, ts_back, samp_sec, note_duration),
        dichotic_pitch(0, peak, width, ts_sig, ts_back, samp_sec, note_duration)
    ]))
dp = np.concatenate(parts)

# Process and play as before
#dp = lowpass(dp, f_max/f_nyquist)
dp = cos_window(dp, round(samp_sec/100))
dp = dp / np.max(np.abs(dp))
sf.write('dpSBR.wav', dp, samp_sec)
Audio(dp.T, rate=samp_sec)

In [ ]:
# Dichotic Pitch melody
class Note:
    def __init__(self, peak=0, ts_sig=0.6, duration=0.1):
        self.peak = peak
        self.ts_sig = ts_sig
        self.duration = duration
        self.sbr = 1
        self.width = np.array(peak)/20 if isinstance(peak, (list, np.ndarray)) else peak/20
        self.ts_back = -0.6

# Define the melody
notes = []
notes.append(Note())  # blank interval
notes.append(Note(peak=[300, 600, 900], duration=0.6))
notes.append(Note())  # blank
notes.append(Note(peak=[400, 800], duration=0.6))
notes.append(Note())  # blank
notes.append(Note(peak=[300, 600, 900], duration=0.6))
notes.append(Note())  # blank
notes.append(Note(peak=[200, 400, 600, 800], duration=0.6))
notes.append(Note())  # blank

# Build the sound vector
parts = []
for note in notes:
    parts.append(dichotic_pitch(note.sbr, note.peak, note.width, 
                      note.ts_sig, note.ts_back, samp_sec, note.duration))
dp = np.concatenate(parts)

# Process and play as before
#dp = lowpass(dp, f_max/f_nyquist)
dp = cos_window(dp, round(samp_sec/100))
dp = dp/np.max(np.abs(dp))
sf.write('dpMelody.wav', dp, samp_sec)
Audio(dp.T, rate=samp_sec)
